In [1]:
# Define project information

import sys

PROJECT_ID = "gen-lang-client-0303567819"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: gen-lang-client-0303567819


Your active configuration is: [personal]


In [5]:
import sys

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [21]:
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from IPython.display import Markdown, display
from vertexai.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part,
)
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import MultiModalEmbeddingModel

In [8]:
# Multimodal models: Choose based on your performance/cost needs

multimodal_model_2_0_flash = GenerativeModel(
    "gemini-2.0-flash-001"
) # Gemini latest Gemini 2.0 Flash Model

multimodal_model_15 = GenerativeModel(
    "gemini-1.5-pro-001"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - complex reasoning

# Multimodal models: Choose based on your performance/cost needs
multimodal_model_15_flash = GenerativeModel(
    "gemini-1.5-flash-001"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - faster inference

# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")

# Load multimodal embedding model from pre-trained source
multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained(
    "multimodalembedding@001"
)  # works with image, image with caption(~32 words), video, video with caption(~32 words)

E0000 00:00:1762613845.227259   13116 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


RetryError: Timeout of 120.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

In [22]:
# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")

E0000 00:00:1762615718.527455   13116 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


RetryError: Timeout of 120.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

In [19]:
multimodal_embedding_model = GenerativeModel("gemini-2.5-flash")

/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [20]:
from multimodal_qa_with_rag_utils import (
    get_document_metadata,
    set_global_variable,
)

set_global_variable("text_embedding_model", text_embedding_model)
set_global_variable("multimodal_embedding_model", multimodal_embedding_model)

NameError: name 'text_embedding_model' is not defined

In [14]:
from multimodal_qa_with_rag_utils import (
    get_document_metadata,
    set_global_variable,
)

set_global_variable("text_embedding_model", text_embedding_model)
set_global_variable("multimodal_embedding_model", multimodal_embedding_model)

NameError: name 'text_embedding_model' is not defined

In [9]:
# CÉLULA 78 (OPCIONAL) - 📄 EXTRAIR MAIS IMAGENS DO PDF PARA COMPARAÇÃO
# Esta célula extrai imagens do map.pdf para ter mais dados para comparação

print("=== EXTRAINDO IMAGENS DO PDF PARA AMPLIAR DATASET ===\n")

import fitz  # PyMuPDF
import os

def extrair_imagens_do_pdf(pdf_path, output_dir="images/", prefixo="map"):
    """
    Extrai imagens de um PDF e salva na pasta de imagens
    """
    print(f"🔍 Processando PDF: {pdf_path}")
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF não encontrado: {pdf_path}")
        return []
    
    # Criar diretório se não existir
    os.makedirs(output_dir, exist_ok=True)
    
    # Abrir PDF
    doc = fitz.open(pdf_path)
    imagens_extraidas = []
    
    print(f"📊 PDF tem {len(doc)} páginas")
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        images = page.get_images()
        
        print(f"📄 Página {page_num + 1}: {len(images)} imagens encontradas")
        
        for img_index, img in enumerate(images):
            try:
                # Extrair imagem
                xref = img[0]
                pix = fitz.Pixmap(doc, xref)
                
                # Converter para RGB se necessário
                if pix.colorspace and pix.colorspace.n > 3:
                    pix = fitz.Pixmap(fitz.csRGB, pix)
                
                # Nome do arquivo
                img_filename = f"{prefixo}_page_{page_num + 1}_img_{img_index + 1}.png"
                img_path = os.path.join(output_dir, img_filename)
                
                # Salvar imagem
                pix.save(img_path)
                imagens_extraidas.append(img_path)
                
                print(f"  ✅ Extraída: {img_filename}")
                
                pix = None  # Liberar memória
                
            except Exception as e:
                print(f"  ❌ Erro ao extrair imagem {img_index}: {e}")
                continue
    
    doc.close()
    print(f"\n🎉 Total de {len(imagens_extraidas)} imagens extraídas!")
    return imagens_extraidas

# Verificar quantas imagens temos atualmente
current_images = len([f for f in os.listdir("images/") if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
print(f"📊 Imagens atuais na pasta: {current_images}")

if current_images <= 1:
    print("🔄 Extraindo imagens do PDF para ter mais dados...")
    
    # Extrair do map.pdf se existir
    if os.path.exists("map/map.pdf"):
        imagens_extraidas = extrair_imagens_do_pdf("map/map.pdf", "images/", "map")
        
        if imagens_extraidas:
            print(f"\n✅ {len(imagens_extraidas)} novas imagens adicionadas!")
            print("🚀 Agora execute a CÉLULA 76 novamente para processar todas as imagens")
            print("   Depois execute a CÉLULA 70 para testar similaridade com mais dados")
        else:
            print("❌ Nenhuma imagem foi extraída do PDF")
    else:
        print("❌ Arquivo map/map.pdf não encontrado")
        
        # Verificar outros PDFs disponíveis
        print("\n🔍 Procurando outros PDFs...")
        pdf_paths = []
        for root, dirs, files in os.walk("."):
            for file in files:
                if file.lower().endswith('.pdf'):
                    pdf_paths.append(os.path.join(root, file))
        
        if pdf_paths:
            print("📋 PDFs encontrados:")
            for i, pdf_path in enumerate(pdf_paths[:3], 1):  # Mostrar apenas os 3 primeiros
                print(f"  {i}. {pdf_path}")
                
            # Processar o primeiro PDF encontrado
            if pdf_paths:
                primeiro_pdf = pdf_paths[0]
                print(f"\n🔄 Processando: {primeiro_pdf}")
                imagens_extraidas = extrair_imagens_do_pdf(primeiro_pdf, "images/", "doc")
                
                if imagens_extraidas:
                    print(f"\n✅ {len(imagens_extraidas)} imagens extraídas de {primeiro_pdf}!")
                    print("🚀 Execute a CÉLULA 76 novamente para processar todas as imagens")
        else:
            print("❌ Nenhum PDF encontrado para extrair imagens")
            
else:
    print("✅ Já há múltiplas imagens na pasta")
    print("Execute a CÉLULA 76 para processar todas e depois a CÉLULA 70 para testar similaridade")

# Mostrar status final
final_images = len([f for f in os.listdir("images/") if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
print(f"\n📊 STATUS FINAL: {final_images} imagens na pasta 'images/'")

if final_images > 1:
    print("🎉 Pronto para testar busca por similaridade!")
    print("📋 PRÓXIMOS PASSOS:")
    print("  1. Execute CÉLULA 76 (processar todas as imagens)")
    print("  2. Execute CÉLULA 70 (busca por similaridade)")
    print("  3. Execute CÉLULA 71 (análise contextual)")
else:
    print("⚠️  Ainda há apenas 1 imagem. Adicione mais imagens manualmente na pasta 'images/'")


=== EXTRAINDO IMAGENS DO PDF PARA AMPLIAR DATASET ===

📊 Imagens atuais na pasta: 1
🔄 Extraindo imagens do PDF para ter mais dados...
🔍 Processando PDF: map/map.pdf
📊 PDF tem 1 páginas
📄 Página 1: 0 imagens encontradas

🎉 Total de 0 imagens extraídas!
❌ Nenhuma imagem foi extraída do PDF

📊 STATUS FINAL: 1 imagens na pasta 'images/'
⚠️  Ainda há apenas 1 imagem. Adicione mais imagens manualmente na pasta 'images/'


In [10]:
# CÉLULA 75 (NOVO) - 📂 PROCESSAMENTO DIRETO DE IMAGENS DA PASTA
# Função para ler todas as imagens da pasta images/ e gerar embeddings para RAG

import os
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from multimodal_qa_with_rag_utils import (
    get_image_embedding_from_multimodal_embedding_model,
    get_gemini_response
)

def processar_imagens_da_pasta(
    pasta_imagens="images/",
    embedding_size=512,
    gerar_descricoes=True,
    formatos_suportados=['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
):
    """
    Processa todas as imagens de uma pasta, gerando embeddings e descrições para RAG
    
    Args:
        pasta_imagens: Caminho da pasta com imagens
        embedding_size: Tamanho do embedding (128, 256, 512, 1408)
        gerar_descricoes: Se deve gerar descrições das imagens com Gemini
        formatos_suportados: Lista de formatos de imagem aceitos
    
    Returns:
        pd.DataFrame: DataFrame compatível com o sistema RAG existente
    """
    print(f"🔍 PROCESSANDO IMAGENS DA PASTA: {pasta_imagens}")
    print("="*60)
    
    # Verificar se a pasta existe
    if not os.path.exists(pasta_imagens):
        print(f"❌ Pasta '{pasta_imagens}' não encontrada!")
        return pd.DataFrame()
    
    # Encontrar todas as imagens na pasta
    imagens_encontradas = []
    for formato in formatos_suportados:
        pattern = os.path.join(pasta_imagens, f"*{formato}")
        imagens_encontradas.extend(glob.glob(pattern))
        pattern = os.path.join(pasta_imagens, f"*{formato.upper()}")
        imagens_encontradas.extend(glob.glob(pattern))
    
    # Remover duplicatas
    imagens_encontradas = list(set(imagens_encontradas))
    
    if not imagens_encontradas:
        print(f"❌ Nenhuma imagem encontrada na pasta '{pasta_imagens}'")
        print(f"Formatos suportados: {formatos_suportados}")
        return pd.DataFrame()
    
    print(f"📊 Encontradas {len(imagens_encontradas)} imagens:")
    for img in imagens_encontradas:
        print(f"  - {os.path.basename(img)}")
    
    # Lista para armazenar dados processados
    dados_imagens = []
    
    # Prompt para descrição das imagens
    prompt_descricao = """Analise esta imagem detalhadamente e forneça uma descrição precisa.
    Inclua:
    - O que você vê na imagem
    - Elementos principais e detalhes importantes
    - Texto visível (se houver)
    - Tipo de imagem (mapa, diagrama, foto, etc.)
    - Informações relevantes para busca e recuperação
    
    Seja específico e detalhado para facilitar buscas futuras."""
    
    print(f"\n🚀 PROCESSANDO CADA IMAGEM...")
    print("="*60)
    
    for i, caminho_imagem in enumerate(imagens_encontradas, 1):
        nome_arquivo = os.path.basename(caminho_imagem)
        print(f"\n📸 PROCESSANDO {i}/{len(imagens_encontradas)}: {nome_arquivo}")
        
        try:
            # 1. Gerar embedding da imagem
            print("  🔄 Gerando embedding...")
            image_embedding = get_image_embedding_from_multimodal_embedding_model(
                image_uri=caminho_imagem,
                embedding_size=embedding_size,
                return_array=True
            )
            print(f"  ✅ Embedding gerado: shape {image_embedding.shape}")
            
            # 2. Gerar descrição da imagem (se solicitado)
            descricao = ""
            if gerar_descricoes:
                print("  🤖 Gerando descrição com Gemini...")
                try:
                    from vertexai.generative_models import Image as GeminiImage
                    imagem_gemini = GeminiImage.load_from_file(caminho_imagem)
                    
                    descricao = get_gemini_response(
                        multimodal_model_2_0_flash,
                        model_input=[prompt_descricao, imagem_gemini],
                        stream=False,
                    )
                    print(f"  ✅ Descrição gerada: {len(descricao)} caracteres")
                    
                except Exception as desc_error:
                    print(f"  ⚠️  Erro ao gerar descrição: {desc_error}")
                    descricao = f"Imagem: {nome_arquivo}"
            
            # 3. Gerar embedding da descrição (para compatibilidade com RAG)
            text_embedding = None
            if descricao:
                try:
                    from multimodal_qa_with_rag_utils import get_text_embedding_from_text_embedding_model
                    text_embedding = get_text_embedding_from_text_embedding_model(descricao)
                    print("  ✅ Text embedding da descrição gerado")
                except Exception as text_emb_error:
                    print(f"  ⚠️  Erro ao gerar text embedding: {text_emb_error}")
            
            # 4. Criar registro compatível com o sistema existente
            registro = {
                'file_name': f"pasta_images_{nome_arquivo}",  # Nome único
                'page_num': 1,  # Imagens individuais = página 1
                'img_num': i,
                'img_path': caminho_imagem,
                'img_desc': descricao,
                'mm_embedding_from_img_only': image_embedding.tolist(),  # Compatibilidade
                'text_embedding_from_image_description': text_embedding if text_embedding else None,
                'source_type': 'pasta_imagens',  # Identificar origem
                'original_filename': nome_arquivo
            }
            
            dados_imagens.append(registro)
            print(f"  ✅ Processamento concluído para {nome_arquivo}")
            
        except Exception as e:
            print(f"  ❌ Erro ao processar {nome_arquivo}: {e}")
            continue
    
    # Criar DataFrame
    if dados_imagens:
        df_imagens = pd.DataFrame(dados_imagens)
        print(f"\n🎉 PROCESSAMENTO CONCLUÍDO!")
        print(f"📊 DataFrame criado com {len(df_imagens)} imagens processadas")
        print(f"📋 Colunas: {list(df_imagens.columns)}")
        
        return df_imagens
    else:
        print(f"\n❌ Nenhuma imagem foi processada com sucesso")
        return pd.DataFrame()

print("✅ Função 'processar_imagens_da_pasta' criada com sucesso!")


✅ Função 'processar_imagens_da_pasta' criada com sucesso!


In [16]:
# CÉLULA 76 (EXECUTAR) - 🚀 PROCESSAMENTO DAS IMAGENS DA PASTA images/
# Executa o processamento de todas as imagens e cria o image_metadata_df

print("=== PROCESSAMENTO COMPLETO DA PASTA IMAGES/ ===\n")

# Executar o processamento das imagens
try:
    image_metadata_df = processar_imagens_da_pasta(
        pasta_imagens="images/",
        embedding_size=512,
        gerar_descricoes=True,  # Gerar descrições detalhadas com Gemini
        formatos_suportados=['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
    )
    
    if not image_metadata_df.empty:
        print(f"\n🎉 SUCESSO TOTAL!")
        print(f"📊 image_metadata_df criado com {len(image_metadata_df)} imagens")
        
        # Mostrar resumo das imagens processadas
        print(f"\n📋 RESUMO DAS IMAGENS PROCESSADAS:")
        print("="*50)
        for idx, row in image_metadata_df.iterrows():
            print(f"\n🖼️  Imagem {idx + 1}:")
            print(f"  📁 Arquivo: {row['original_filename']}")
            print(f"  📂 Caminho: {row['img_path']}")
            print(f"  📊 Embedding shape: {len(row['mm_embedding_from_img_only'])}")
            
            # Mostrar início da descrição
            desc = row['img_desc']
            if desc and len(desc) > 10:
                print(f"  📝 Descrição: {desc[:150]}{'...' if len(desc) > 150 else ''}")
        
        # Verificar compatibilidade com sistema RAG existente
        print(f"\n✅ COMPATIBILIDADE COM SISTEMA RAG:")
        colunas_necessarias = ['img_path', 'mm_embedding_from_img_only', 'img_desc', 'file_name', 'page_num']
        for col in colunas_necessarias:
            if col in image_metadata_df.columns:
                print(f"  ✅ {col}: OK")
            else:
                print(f"  ❌ {col}: FALTANDO")
        
        # Salvar para uso futuro (opcional)
        try:
            image_metadata_df.to_pickle("image_metadata_from_folder.pkl")
            print(f"\n💾 DataFrame salvo em 'image_metadata_from_folder.pkl'")
        except Exception as save_error:
            print(f"\n⚠️  Não foi possível salvar: {save_error}")
        
        print(f"\n🚀 PRÓXIMOS PASSOS:")
        print(f"1. Agora você pode executar a CÉLULA 70 (Validação)")
        print(f"2. Depois executar a CÉLULA 71 (Análise Contextual)")
        print(f"3. O sistema RAG está pronto para perguntas sobre as imagens!")
        
    else:
        print(f"\n❌ FALHA: Nenhuma imagem foi processada")
        print(f"Verifique se:")
        print(f"- A pasta 'images/' existe")
        print(f"- Há imagens válidas na pasta")
        print(f"- Os modelos estão carregados corretamente")

except Exception as e:
    print(f"❌ ERRO NO PROCESSAMENTO: {e}")
    import traceback
    traceback.print_exc()
    
    print(f"\n💡 POSSÍVEIS SOLUÇÕES:")
    print(f"- Verifique se os modelos estão carregados")
    print(f"- Verifique se a pasta 'images/' existe")
    print(f"- Execute as células de setup dos modelos primeiro")


=== PROCESSAMENTO COMPLETO DA PASTA IMAGES/ ===

🔍 PROCESSANDO IMAGENS DA PASTA: images/
📊 Encontradas 1 imagens:
  - B2_room.jpeg

🚀 PROCESSANDO CADA IMAGEM...

📸 PROCESSANDO 1/1: B2_room.jpeg
  🔄 Gerando embedding...
  ❌ Erro ao processar B2_room.jpeg: 'GenerativeModel' object has no attribute 'get_embeddings'

❌ Nenhuma imagem foi processada com sucesso

❌ FALHA: Nenhuma imagem foi processada
Verifique se:
- A pasta 'images/' existe
- Há imagens válidas na pasta
- Os modelos estão carregados corretamente


/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/vision_models/_vision_models.py:153: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [14]:
# CELL 70 (STEP 3) - 🎯 VALIDATION: Similarity search with M3.jpeg in the dataset
# This cell uses the image_metadata_df created by Cell 76 to search for similar images

print("=== M3.JPEG SIMILARITY SEARCH VALIDATION ===\n")

# Check if we have the image metadata DataFrame
if 'image_metadata_df' not in locals():
    print("❌ image_metadata_df not found!")
    print("\n💡 To resolve:")
    print("1. Execute CELL 78 first (Extract images from PDF)")
    print("2. Execute CELL 76 next (Process Images)")
    print("3. Then execute this cell again")
    
elif image_metadata_df.empty:
    print("❌ image_metadata_df is empty!")
    print("Execute CELL 76 to process images from the folder")
    
else:
    print("✅ image_metadata_df available!")
    print(f"📊 Dataset: {len(image_metadata_df)} images processed")
    
    # Show all images in the dataset
    print(f"\n📋 IMAGES IN DATASET:")
    for idx, row in image_metadata_df.iterrows():
        print(f"  {idx + 1}. {row['original_filename']}")
    
    # Find M3.jpeg in the dataset
    m3_rows = image_metadata_df[image_metadata_df['original_filename'].str.contains('M3.jpeg', case=False, na=False)]
    
    if m3_rows.empty:
        print("\n❌ M3.jpeg not found in dataset!")
        print("Check if the image is in the 'images/' folder and execute CELL 76 again")
    else:
        print(f"\n✅ M3.jpeg found in dataset!")
        m3_row = m3_rows.iloc[0]
        print(f"  📁 File: {m3_row['original_filename']}")
        print(f"  📂 Path: {m3_row['img_path']}")
        
        # Extract M3.jpeg embedding
        m3_embedding = np.array(m3_row['mm_embedding_from_img_only'])
        print(f"  📊 Embedding shape: {m3_embedding.shape}")
        
        # Create dataset without M3.jpeg itself for comparison
        other_images_df = image_metadata_df[~image_metadata_df['original_filename'].str.contains('M3.jpeg', case=False, na=False)]
        
        if other_images_df.empty:
            print("\n⚠️  Only M3.jpeg found in dataset")
            print("Execute CELL 78 first to extract more images from PDF")
            print("Then execute CELL 76 to process all images")
        else:
            print(f"\n🔍 EXECUTING SIMILARITY SEARCH...")
            print(f"📊 Comparing M3.jpeg with {len(other_images_df)} other images")
            
            # Use our robust alternative function
            try:
                similar_results = buscar_imagens_similares_com_embedding(
                    image_embedding=m3_embedding,
                    image_metadata_df=other_images_df,
                    top_n=min(5, len(other_images_df))  # Top N most similar images
                )
                
                if similar_results:
                    print(f"\n🎉 SUCCESS! Found {len(similar_results)} images similar to M3.jpeg:")
                    print("="*80)
                    
                    for i, result in enumerate(similar_results, 1):
                        print(f"\n🖼️  RESULT {i}:")
                        print(f"  📈 Similarity: {result['cosine_score']:.4f}")
                        print(f"  📁 File: {result['file_name']}")
                        print(f"  📂 Path: {result['img_path']}")
                        
                        # Show description if available
                        desc = result['img_desc']
                        if desc and desc != 'N/A' and len(str(desc)) > 10:
                            desc_str = str(desc)
                            print(f"  📝 Description: {desc_str[:200]}{'...' if len(desc_str) > 200 else ''}")
                    
                    # Save results for later use
                    globals()['matching_results'] = similar_results
                    print(f"\n💾 Results saved to 'matching_results' variable")
                    
                    # Similarity score analysis
                    scores = [r['cosine_score'] for r in similar_results]
                    print(f"\n📊 SCORE ANALYSIS:")
                    print(f"  - Maximum score: {max(scores):.4f}")
                    print(f"  - Minimum score: {min(scores):.4f}")
                    print(f"  - Average score: {sum(scores)/len(scores):.4f}")
                    
                    print(f"\n✅ VALIDATION COMPLETED SUCCESSFULLY!")
                    print(f"🚀 Now you can execute CELL 71 for contextual analysis!")
                    
                else:
                    print("❌ No similar images found.")
                    print("This may indicate problems with embeddings or very different data.")
                    
            except Exception as e:
                print(f"❌ Error during search: {e}")
                import traceback
                traceback.print_exc()
                
                print(f"\n💡 DIAGNOSIS:")
                print(f"- Check if function 'buscar_imagens_similares_com_embedding' was defined (CELL 75)")
                print(f"- Check if there are other images besides M3.jpeg in the dataset")


=== M3.JPEG SIMILARITY SEARCH VALIDATION ===

✅ image_metadata_df available!
📊 Dataset: 1 images processed

📋 IMAGES IN DATASET:
  1. B2_room.jpeg

❌ M3.jpeg not found in dataset!
Check if the image is in the 'images/' folder and execute CELL 76 again


In [15]:
# Novo Código

# CÉLULA 71 (PASSO 4) - 🤖 ANÁLISE CONTEXTUAL: Perguntas sobre M3.jpeg com base nas imagens similares
# Esta célula usa os resultados da busca por similaridade para análise contextual

print("=== ANÁLISE CONTEXTUAL DA M3.JPEG ===\n")

# Verificar se temos resultados da busca anterior
if 'matching_results' not in locals() or not matching_results:
    print("❌ Nenhum resultado de busca por similaridade encontrado.")
    print("Execute a célula anterior (70) primeiro para obter os resultados.")
    
    print("\n💡 Como alternativa, vou fazer uma análise direta da M3.jpeg:")
    
    try:
        from vertexai.generative_models import Image as GeminiImage
        m3_image = GeminiImage.load_from_file("images/M3.jpeg")
        
        pergunta_direta = """Analise esta imagem detalhadamente. 
        O que você vê? Descreva todos os elementos visíveis.
        Se é um mapa ou planta, identifique os elementos principais."""
        
        resposta_direta = get_gemini_response(
            multimodal_model_2_0_flash,
            model_input=[pergunta_direta, m3_image],
            stream=False,
        )
        
        print(f"🤖 ANÁLISE DIRETA DA M3.JPEG:")
        print(f"{resposta_direta}")
        
    except Exception as direct_error:
        print(f"❌ Erro na análise direta: {direct_error}")

else:
    print(f"✅ Temos {len(matching_results)} resultados de busca por similaridade!")
    
    # Preparar contexto baseado nos resultados similares
    contexto_descricoes = []
    contexto_caminhos = []
    
    for i, result in enumerate(matching_results):
        desc = result.get('img_desc', '')
        caminho = result.get('img_path', '')
        arquivo = result.get('file_name', '')
        score = result.get('cosine_score', 0)
        
        if desc and desc != 'N/A' and len(str(desc)) > 10:
            contexto_descricoes.append(f"Imagem similar {i+1} (similaridade: {score:.3f}): {desc}")
        
        if caminho and caminho != 'N/A':
            contexto_caminhos.append(caminho)
    
    print(f"📝 Coletadas {len(contexto_descricoes)} descrições de imagens similares")
    
    # Perguntas específicas sobre a M3.jpeg
    perguntas_contextualizadas = [
        "Baseado nas imagens similares encontradas, o que você pode me dizer sobre a M3.jpeg?",
        "Quais elementos em comum existem entre a M3.jpeg e as imagens similares?",
        "Se a M3.jpeg é um mapa ou planta, quais informações específicas posso extrair?",
        "Há algum padrão arquitetônico ou de layout visível na M3.jpeg?",
        "What are the rooms in this floor? (baseado no contexto das imagens similares)"
    ]
    
    print("\n🤖 ANÁLISE CONTEXTUAL COM GEMINI:")
    
    try:
        # Carregar a imagem M3.jpeg
        from vertexai.generative_models import Image as GeminiImage
        m3_image = GeminiImage.load_from_file("images/M3.jpeg")
        
        # Preparar contexto das imagens similares
        contexto_texto = "\n".join(contexto_descricoes[:3])  # Top 3 descrições
        
        for i, pergunta in enumerate(perguntas_contextualizadas, 1):
            print(f"\n" + "="*70)
            print(f"📋 PERGUNTA {i}: {pergunta}")
            print("="*70)
            
            # Criar prompt contextualizado
            prompt_contextualizado = f"""
            Analise a imagem fornecida considerando o seguinte contexto de imagens similares:
            
            CONTEXTO DE IMAGENS SIMILARES ENCONTRADAS:
            {contexto_texto}
            
            PERGUNTA ESPECÍFICA:
            {pergunta}
            
            Por favor, forneça uma resposta detalhada baseada tanto na análise visual da imagem 
            quanto no contexto das imagens similares fornecido acima.
            """
            
            try:
                resposta = get_gemini_response(
                    multimodal_model_2_0_flash,
                    model_input=[prompt_contextualizado, m3_image],
                    stream=False,
                )
                
                print(f"🤖 RESPOSTA CONTEXTUALIZADA:")
                print(f"{resposta}")
                
            except Exception as gemini_error:
                print(f"❌ Erro na análise contextual: {gemini_error}")
                
                # Fallback: análise simples sem contexto
                try:
                    resposta_simples = get_gemini_response(
                        multimodal_model_2_0_flash,
                        model_input=[pergunta, m3_image],
                        stream=False,
                    )
                    print(f"🤖 RESPOSTA SIMPLES (sem contexto):")
                    print(f"{resposta_simples}")
                    
                except Exception as simple_error:
                    print(f"❌ Erro na análise simples: {simple_error}")
    
    except Exception as e:
        print(f"❌ Erro ao carregar imagem: {e}")
    
    # Mostrar resumo final
    print(f"\n" + "="*70)
    print("📊 RESUMO DOS RESULTADOS DE SIMILARIDADE:")
    print("="*70)
    
    for i, result in enumerate(matching_results, 1):
        print(f"\n🖼️  Imagem Similar {i}:")
        print(f"  📈 Similaridade: {result.get('cosine_score', 0):.4f}")
        print(f"  📁 Arquivo: {result.get('file_name', 'N/A')}")
        print(f"  📄 Página: {result.get('page_num', 'N/A')}")
        print(f"  📂 Caminho: {result.get('img_path', 'N/A')}")

print(f"\n✅ ANÁLISE CONTEXTUAL CONCLUÍDA!")
print(f"A M3.jpeg foi analisada usando o contexto das imagens similares encontradas via embedding.")


=== ANÁLISE CONTEXTUAL DA M3.JPEG ===

❌ Nenhum resultado de busca por similaridade encontrado.
Execute a célula anterior (70) primeiro para obter os resultados.

💡 Como alternativa, vou fazer uma análise direta da M3.jpeg:
❌ Erro na análise direta: [Errno 2] No such file or directory: 'images/M3.jpeg'

✅ ANÁLISE CONTEXTUAL CONCLUÍDA!
A M3.jpeg foi analisada usando o contexto das imagens similares encontradas via embedding.


In [16]:
# CELL 72 - 🤖 DIRECT M3.JPEG ANALYSIS WITH GEMINI (CORRECTED)
# Corrected function to ask for figure details using the Gemini model

print("=== DIRECT M3.JPEG ANALYSIS WITH GEMINI ===\n")

def ask_figure_details_corrected(model, image_path=None):
    """
    Automatically asks for figure details from the multimodal Gemini model.
    Args:
        model: loaded Gemini model
        image_path: path to the image (string)
    """
    print("🔍 Analyzing image with Gemini...")
    
    # Specific questions about the image
    questions = [
        "What are the rooms or areas shown in this floor plan?",
        "How can I go from room 2001 to room 2037?"
    ]
    try:
        # Load the image
        from vertexai.generative_models import Image as GeminiImage
        image_object = GeminiImage.load_from_file(image_path)
        print(f"✅ Image loaded: {image_path}")
        
        # Ask each question
        for i, question in enumerate(questions, 1):
            print(f"\n📋 QUESTION {i}: {question}")
            print("-" * 60)
            
            try:
                # Use the model directly (most reliable method)
                response = model.generate_content([question, image_object])
                response_text = response.text if hasattr(response, 'text') else str(response)
                
                print(f"🤖 RESPONSE:")
                print(f"{response_text}")
                
            except Exception as question_error:
                print(f"❌ Error in question {i}: {question_error}")
                
                # Try alternative method with get_gemini_response
                try:
                    alt_response = get_gemini_response(
                        model,
                        model_input=[question, image_object],
                        stream=False
                    )
                    print(f"🤖 RESPONSE (alternative method):")
                    print(f"{alt_response}")
                except Exception as alt_error:
                    print(f"❌ Alternative method also failed: {alt_error}")
        
    except Exception as e:
        print(f"❌ General error analyzing image: {e}")
        import traceback
        traceback.print_exc()

# Test the corrected function automatically
try:
    # Check if the model is available
    if 'multimodal_model_2_0_flash' in locals():
        print("✅ Gemini 2.0 Flash model available")
        
        # Test with M3.jpeg
        ask_figure_details_corrected(
            multimodal_model_2_0_flash, 
            image_path="images/B2_room.jpeg"
        )
        
    else:
        print("❌ multimodal_model_2_0_flash model not found")
        print("Execute the model setup cells first")
        
        # Try to load basic model
        try:
            from vertexai.generative_models import GenerativeModel
            temp_model = GenerativeModel("gemini-1.5-flash")
            print("🔄 Using Gemini 1.5 Flash model as alternative...")
            
            ask_figure_details_corrected(
                temp_model,
                image_path="images/M3.jpeg"
            )
        except Exception as model_error:
            print(f"❌ Error loading alternative model: {model_error}")
        
except Exception as e:
    print(f"❌ Error executing analysis: {e}")
    print("Check if:")
    print("- The image 'images/M3.jpeg' exists")
    print("- Vertex AI models are configured")
    print("- Dependencies are installed")

print(f"\n✅ Analysis completed!")


=== DIRECT M3.JPEG ANALYSIS WITH GEMINI ===

✅ Gemini 2.0 Flash model available
🔍 Analyzing image with Gemini...
✅ Image loaded: images/B2_room.jpeg

📋 QUESTION 1: What are the rooms or areas shown in this floor plan?
------------------------------------------------------------
🤖 RESPONSE:
Based on the floor plan provided, here are the rooms or areas that are visible:

*   **Rooms:**
    *   Room 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2010, 2012, 2015, 2025, 2029, 2032, 2034, 2035, 2036, 2037
*   **Amenities:**
    *   Washrooms
    *   Women's Washroom
    *   Women's Accessible Washroom
    *   Accessible Washroom
    *   Men's Washroom
    *   Men's Accessible Washroom
    *   Accessible Shower and Changeroom
    *   Elevator
    *   Stairs
    *   Bus Stop
    *   Bike Parking
    *   Information
    *   Food
    *   Interior and exterior pathways

Other items:

*   T
*   D
*   You are here
*   Exit
*   Adjacent Buildings

📋 QUESTION 2: How can I go from room 2001 to room 2037?
